<a href = https://developers.google.com/maps/documentation/places/web-service/details#PlaceDetailsRequests ><h1>Data Extraction From GOOGLE Place Details API</h1></a>

In [1]:
import requests
import json
import pandas as pd
import numpy as np
import time


In [2]:
apiKey = #INSERT APIKEY

<h2>Functions</h2>

In [3]:
def make_request_place_details(inputs):

    PLACE_ID = 'place_id='+inputs['PLACE_ID']
    FIELDS = 'fields='+','.join(inputs['FIELDS'])
    url = f'https://maps.googleapis.com/maps/api/place/details/json?{PLACE_ID}&{FIELDS}&key={apiKey}'
    r = requests.get(url)
    j = json.loads(r.text)
    
    return j

In [10]:
def create_dictionaries_place_details(j):
    
    results = j['result']
    d_details = {param : [] for param in ['place_id', 'name', 'formatted_phone_number','opening_hours', 'website']}
    
    if results.get('reviews'):
        reviews = results['reviews']
        num_reviews = len(reviews)

        d_reviews = {param : [] for param in ['place_id', 'name', 'text', 'author_name', 'author_url', 'language', 'rating', 'relative_time_description', 'time', 'profile_photo_url']}
 

        for i in range(num_reviews):
            review = reviews[i]
            for param in d_reviews.keys():
                if param == 'place_id':
                    data_to_store = results['place_id']
                elif param == 'name':
                    data_to_store = results['name']
                elif review.get(param):
                    data_to_store = review[param]
                else:
                    data_to_store = None
                d_reviews[param].append(data_to_store)
    else:
        d_reviews = None
        
    for param in d_details.keys():        
        if results.get(param):
            data_to_store = results[param]
        else:
            data_to_store = None
        d_details[param].append(data_to_store)
    
    return d_reviews, d_details

In [11]:
def create_dataframe_place_details(inputs):
    PLACE_IDS = inputs.copy()
    PARAMS = {
        'PLACE_ID' : None,
        'FIELDS': [
                  'name',
                  'place_id',
                  'rating', 
                  'formatted_phone_number',
                  'opening_hours',
                  'reviews',
                  'website']
    }
    dfs_reviews = []
    dfs_details = []
    for p_id in PLACE_IDS:
        PARAMS['PLACE_ID'] = p_id
        j = make_request_place_details(PARAMS)
        d_reviews, d_details = create_dictionaries_place_details(j)
        dfs_reviews.append(pd.DataFrame(d_reviews))
        dfs_details.append(pd.DataFrame(d_details))
        
    df_reviews_merge = pd.concat(dfs_reviews, ignore_index=True)
    df_details_merge = pd.concat(dfs_details, ignore_index=True)
    return df_reviews_merge, df_details_merge

<h2>Input Parameters</h2>

In [ ]:
df = pd.read_csv('data/data_place.csv')
df = df.drop_duplicates(subset = ['place_id'])
place_ids = list(df['place_id'])
num_batches = 10
bounds = np.linspace(0,len(place_ids), num_batches+1).astype(int)

dfs_reviews = []
dfs_details = []
for i in range(num_batches):
    INPUT = place_ids[bounds[i]: bounds[i+1]]
    df_r, df_d = create_dataframe_place_details(INPUT)
    dfs_reviews.append(df_r)
    dfs_details.append(df_d)
    file_reviews = f'data_reviews_{i}.csv'
    file_details = f'data_details_{i}.csv'
    df_r.to_csv(file_reviews)
    df_d.to_csv(file_details)

df_reviews = pd.concat(dfs_reviews, ignore_index=True)
df_details = pd.concat(dfs_details, ignore_index=True)

<h2>Dataframe Reviews</h2>

In [ ]:
df_reviews

<h2>Dataframe details</h2>

In [ ]:
df_details

<h2>Dataframe to csv</h2>

In [ ]:
df_reviews.to_csv('data/data_reviews.csv')
df_details.to_csv('data/data_details.csv')